- 案件：[リサイクル品を扱うネットショップのリサーチと商品登録業務をお手伝い頂けませんか？](https://crowdworks.jp/proposals/193638313#scroll_to_message)
- Notionページ：https://www.notion.so/5364556d13fe44d6afd582e0a49fb29a
### 設計
- ● 検索対象のURLをユーザーに求めるようにする
- ● Selenium ではなく requests と BeautifulSoup を使う方がスピード的に良い？ただし取得料は少ないのでどっちでもいいかも。ポップアップを開いたりするのであれば Selenium が必要<br>
  →　今回はブラウザ操作をするので Selenium が必要
- ▲ Chrome 拡張機能（Resale Trap）を使えるようにできるか？<br>
  →　[【Python】SeleniumでChrome拡張機能を利用する方法](https://self-development.info/%E3%80%90python%E3%80%91selenium%E3%81%A7chrome%E6%8B%A1%E5%BC%B5%E6%A9%9F%E8%83%BD%E3%82%92%E5%88%A9%E7%94%A8%E3%81%99%E3%82%8B%E6%96%B9%E6%B3%95/)　※ ただし、ResaleTrapは Chrome ウェブストアにはないのでこの方法は使え合い<br>
  →　[seleniumのChromeブラウザに普段使ってる拡張機能を追加する](https://techtechfire.com/selenium-python/396/)　※ マニフェストファイルがないとのエラー<br>
  →　ZIP解凍したフォルダ・ファイルでイケた！

In [86]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import pandas as pd

#ブラウザの設定
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# 拡張機能 Resale Trap を有効にする　staff2@polyphony.tokyo／Muzaiko20230606
chrome_options.add_extension(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\extensions/resaletrap_3.0.9.crx')
# Chromeを起動するためのオプションにプロファイルを追加する
chrome_options.add_argument(r'--user-data-dir=C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\profile')

#ブラウザの起動する
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)
browser.implicitly_wait(3)

# ヤフオク商品ページを開く
# url = 'https://page.auctions.yahoo.co.jp/jp/auction/v1094580770' $ 固定入力
print('▼ 対象URLを入力してください')
url_org = input() # 標準入力
browser.get(url_org)
browser.implicitly_wait(3)

C:\Users\amuza\AppData\Local\Temp\ipykernel_2840\1099093426.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)


▼ 対象URLを入力してください


 https://page.auctions.yahoo.co.jp/jp/auction/p1092926375


In [ ]:
# ページを直接開く
print('▼ 対象URLを入力してください')
test_url = input() # 標準入力
browser.get(test_url)

In [79]:
item_name = browser.find_element(By.XPATH,'//h1').text
print('商品名：', item_name)

商品名： SONY　BRAVIA XRJ-65X90J [65インチ] 展示美品 認知特性プロセッサー「XR」搭載のプレミアム4K液晶テレビ FD


In [ ]:
item_price = browser.find_element(By.XPATH,'//dd[@class="Price__value"]').text.replace(' ', '').replace(',', '')
print('価格：', item_price)

In [ ]:
state = browser.find_element(By.XPATH,'//th[contains(text(), "状態")]/following-sibling::td/a').text
print('状態：', state)

In [ ]:
bidnum = browser.find_elements(By.XPATH,'//span[@class="Count__detail"]')[0].text
print('入札数：', bidnum)

In [ ]:
# ・出品者の評価が97％以上
# →　商品ページの出品者欄の「○○%」で判定
seller_rating = browser.find_element(By.XPATH,'//div[@class="Seller__ratingRatio"]').text
print('評価：', seller_rating)

In [ ]:
# 評価数は多い方が望ましい
seller_rating_num = browser.find_element(By.XPATH,'//a[@class="Seller__ratingLink"]').text
print('評価数：', seller_rating_num)

In [ ]:
# ・出品者が「匿名配送」希望でないこと
# →　商品ページのタイトルに「匿名配送」アイコンがないことで判定
try:
    privacy = browser.find_element(By.XPATH,'//span[@class="Icon Icon--privacy"]').text # 匿名配送の場合
except:
    privacy = '匿名配送なし'
print('匿名配送：', privacy)

In [87]:
# 送料の「詳細」ポップアップから配送情報を取得する

# 送料の「詳細」をクリックする
browser.find_element(By.XPATH,'//a[@id="postageDetailCurrent"]').click()
browser.implicitly_wait(1)
# 配送先から「青森県」を選ぶ
# browser.find_element(By.XPATH,'//div[@class="SelectWrap"]//select').click() # 不要
browser.find_element(By.XPATH,'//option[contains(text(), "青森県")]').click()
browser.implicitly_wait(1)

# 配送情報を取得する
try:
    postage_detail_1 = browser.find_element(By.XPATH,'//dt[@class="BidModal__postageName"]').text
    print('配送情報_1：', postage_detail_1)
    postage_detail_2 = browser.find_element(By.XPATH,'//div[@class="BidModal__postagePrice"]').text
    print('配送情報_2：', postage_detail_2)
except:
    pass

# 「✕」をクリックしてポップアップを閉じる
browser.find_element(By.XPATH,'//div[@class="BidModal__box BidModal__box--postage js-modal-box"]//a[contains(text(), "閉じる")]').click()

postage_from = browser.find_element(By.XPATH,'//p[@class="Price__postageFrom"]').text
print('発送元：', postage_from)

send_date = browser.find_element(By.XPATH,'//dt[contains(text(), "発送開始")]/following-sibling::dd').text
print('発送開始：', send_date)

postage = browser.find_element(By.XPATH,'//span[@class="Price__postageValue"]').text
print('送料：', postage)

配送情報_1： 指定業者
配送情報_2： 送料未定
発送元： 発送元：東京都
発送開始： 支払い手続きから3～7日で発送
送料： 落札者負担


In [88]:
# 商品説明文を取得する
# この内容を基に配送業者情報、営業所止め情報、直接取引情報、着払い情報などを取りたい
item_info = browser.find_element(By.XPATH,'//div[@class="ProductExplanation__commentArea"]').text
print('商品説明：', item_info)

商品説明： ご覧頂きまして誠にありがとうございます。 

SONY BRAVIA XRJ-65X90J [65インチ] 展示美品 認知特性プロセッサー「XR」搭載のプレミアム4K液晶テレビです。

落札日より1年のメーカー保証が付きます。
落札額の5％で5年保証にご加入頂けます。

元箱が欠品しています。
美品ですが展示品の為、細かな傷 汚れはご容赦下さい。
視聴に影響するような焼けや傷はありません。

神経質な方の入札は固くお断りいたします。
詳しくはこちらのＨＰをご覧ください。

X95Jシリーズ | テレビ ブラビア | ソニー (sony.jp)

直接引き取りも対応致しますが発送の場合はクロネコヤマト らくらく家財便で発送致します。Dランクです（関東で￥10945+運送保険￥500税込みです）
指定の場所に開梱設置梱包材回収まで料金に含まれます。アンテナ線ブルーレイの接続はオプションで￥3300になります。
送料は、こちらのHPをご覧ください。

https://form.008008.jp/mitumori/PKZI0100Action_doInit.action


離島 遠方 一部発送ができない地区がございます。質問で受付いたしますので確認してからの入札をお願いします。
 

新規の方や評価の悪い方は削除させて頂きます。
恐れ入りますが、ノークレームノーリターンでお願いします。


In [ ]:
# 正規表現での文字列検索テスト
# Pythonで文字列を抽出（位置・文字数、正規表現） 　
# https://note.nkmk.me/python-str-extract/
# 【Python】文字列を検索・抽出する方法【in演算子、findメソッド、正規表現】
# https://python-academia.com/python-str-extract/

import re

s = '直接取引限定　直接取り引き限定　直接取引のみ　直接引き取りのみ　要直接引き渡し　直接取引可能　営業所止め限定　営業所止め可能　営業所止め不可　着払い　着払い希望　着払い可能'

print(re.findall('直接.?.?.?.?.?.?', s))
print(re.findall('営業所.?.?.?.?', s))
print(re.findall('着払い.?.?.?.?', s))

print('直接取引限定：', '直接取引限定' in s)
print('営業所止め：', '営業所止め' in s)
print('着払い：', '着払い' in s)

In [100]:
# ・直接引き取り限定でないこと
# →　商品タイトルおよび商品説明文に「直接取引のみ」などの単語がないことで判定
# →　■ テキストマイニングを活用するとよい？？？

# 商品名判定
print('直接取引関連(商品名)：', re.findall('直接.{10}', item_name))

# 商品説明判定
print('直接取引関連(商品説明)：', re.findall('直接.{10}', item_info))
print('手渡し関連(商品説明)', re.findall('手渡.{10}', item_info))

# print('直接取引(商品説明)：', '直接取引' in item_info)
# print('直接引取(商品説明)：', '直接引取' in item_info)

直接取引関連(商品名)： []
直接取引関連(商品説明)： ['直接引き取りも対応致しま']
手渡し関連(商品説明) []
直接取引(商品説明)： False
直接引取(商品説明)： False


In [101]:
# ・配送が着払いでないこと
# →　「送料負担」が「出品者」であること OR 配送情報詳細や商品説明に「着払い」とないこと
postage_due = browser.find_element(By.XPATH,'//dt[contains(text(), "送料負担")]/following-sibling::dd').text
print('送料負担：', postage_due)

# 商品名判定
print('直接取引関連(商品名)：', re.findall('着払.{10}', item_name))

# 商品説明判定
print('着払(商品説明)：', re.findall('着払.{10}', item_info))

# print('着払(商品説明)：', '着払' in item_info)

送料負担： 落札者
直接取引関連(商品名)： []
着払(商品説明)： []
着払(商品説明)： False


In [104]:
# ・配送がヤマト、佐川、日本郵便などであること（直接エンドユーザーまで届けられること、一部運送会社だと営業所留めがあるため）
# ・営業所止めでないこと
# →　商品説明文に「営業所度止め」などの単語がないことで判定
# 「らくらく家財便」だと安心

# 商品名判定
print('ヤマト(商品名)：', re.findall('ヤマト.{5}', item_name))

# 商品説明判定
print('ヤマト関連(商品説明)：', re.findall('.{4}ヤマト.{5}', item_info))
print('らくらく家財便(商品説明)：', re.findall('らくらく.{10}', item_info))
print('佐川関連(商品説明)', re.findall('佐川.{5}', item_info))
print('日本郵便関連(商品説明)', re.findall('日本郵便.{5}', item_info))
print('営業所止め関連(商品説明)', re.findall('.{5}止め.{5}', item_info))

# print('直接取引(商品説明)：', '直接取引' in item_info)
# print('直接引取(商品説明)：', '直接引取' in item_info)
# print('営業所止め：', '営業所止め' in item_info)

ヤマト(商品名)： []
ヤマト関連(商品説明)： ['クロネコヤマト らくらく']
らくらく家財便(商品説明)： ['らくらく家財便で発送致します']
佐川関連(商品説明) []
日本郵便関連(商品説明) []
営業所止め関連(商品説明) []
直接取引(商品説明)： False
直接引取(商品説明)： False
営業所止め： False


In [ ]:
# オークションの残り日数が一定数あること
# →　商品ページ内の残り時間やオークション終了用低日時で判定
auc_timeleft = browser.find_elements(By.XPATH,'//span[@class="Count__detail"]')[1].text
print('残り日数(画面表示)：', auc_timeleft)

auc_enddatetime = browser.find_element(By.XPATH,'//th[contains(text(), "終了日時")]/following-sibling::td').text
print('終了予定(str型)：', auc_enddatetime)

auc_enddate = auc_enddatetime.split('（')[0]
auc_endtime = auc_enddatetime.split('）')[1]
auc_end = auc_enddate + ' ' + auc_endtime
auc_enddatetime = datetime.datetime.strptime(auc_end, '%Y.%m.%d %H:%M')
print('終了予定(datetime型)：', auc_enddatetime)
# print(type(auc_enddatetime))

dt_now = datetime.datetime.now()
print('現在時刻：', dt_now)

auc_time_left = auc_enddatetime - dt_now
print('残り時間(詳細計算)：', auc_time_left)

In [ ]:
# アマゾンでの販売価格が適切に収まること
# →　新品価格を下回っていることで判定


In [ ]:
# ヤフオクとの価格差がある程度あること（利益が取れること）
# →　ResaleTrapで判定するため本プログラムでは対応不要


In [ ]:
# 取得した情報を CSV 出力用にまとめる
data_list = []

data = {
    '商品名': item_name,
    '価格': item_price,
    '商品説明': item_info,
    '評価': seller_rating,
    '評価数': seller_rating_num,
    '匿名配送': privacy,
    '送料': postage_1,
    '送料詳細_1': postage_detail_1,
    '送料詳細_2': postage_detail_2,
    '送料詳細_3': postage_detail_3,
    '取得日時': datetime.datetime.now().strftime('%Y/%m/%d %H:%M'),
    '終了予定': auc_enddatetime,
    '残り時間': auc_time_left,
    }

data_list.append(data)
print('data_list', data_list)

In [ ]:
# 取得した情報を CSV ファイルに出力する
output_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
csv_file_name = 'ヤフオクtoアマゾンデータ_' + output_time + '.csv'
pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')

# 完成版

In [ ]:

def xxx():
    
def yyy():
    
# csvファイルに出力する
def output():
    output_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    csv_file_name = 'ヤフオクtoアマゾンデータ_' + output_time + '.csv'
    pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')

def main():
    xxx()
    yyy()
    output()

if __name__ == '__main__':
    main()